In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir(r'C:\MSBA\Fall 2020\INSY 5376 - Big Data\Assignments\3')

In [2]:
import pandas as pd
import numpy as np

# set below option to review complete DataFrame column width in output cells
pd.set_option('max_colwidth', -1)

name = 'us_data_subset.xlsx'
df = pd.read_excel(name)
df_copy = df.copy()

df[['TI','AB']] = df[['TI','AB']].fillna('') 
df['TI+AB'] = df['TI'] + ' ' + df['AB']
df.head()

,TI,AB,SO,TI+AB
0,The Role of Business Intelligence and Communication Technologies in Organizational Agility: A Configurational Approach,"This study examines the role that business intelligence (BI) and communication technologies play in how firms may achieve organizational sensing agility, decision making agility, and acting agility in different organizational and environmental contexts. Based on the information-processing view of organizations and dynamic capability theory, we suggest a configurational analytic framework that departs from the standard linear paradigm to examine how IT's effect on agility is embedded in a configuration of organizational and environmental elements. In line with this approach, we use fuzzy-set qualitative comparative analysis (fsQCA) to analyze field survey data from diverse industries. Our findings suggest equifinal pathways to organizational agility and the specific boundary conditions of our middle-range theory that determine what role BI and communication technologies play in organizations' achieving organizational agility. We discuss implications for theory and practice and discuss future research avenues.",JOURNAL OF THE ASSOCIATION FOR INFORMATION SYSTEMS,"The Role of Business Intelligence and Communication Technologies in Organizational Agility: A Configurational Approach This study examines the role that business intelligence (BI) and communication technologies play in how firms may achieve organizational sensing agility, decision making agility, and acting agility in different organizational and environmental contexts. Based on the information-processing view of organizations and dynamic capability theory, we suggest a configurational analytic framework that departs from the standard linear paradigm to examine how IT's effect on agility is embedded in a configuration of organizational and environmental elements. In line with this approach, we use fuzzy-set qualitative comparative analysis (fsQCA) to analyze field survey data from diverse industries. Our findings suggest equifinal pathways to organizational agility and the specific boundary conditions of our middle-range theory that determine what role BI and communication technologies play in organizations' achieving organizational agility. We discuss implications for theory and practice and discuss future research avenues."
1,"OPERATIONAL IT FAILURES, IT VALUE DESTRUCTION, AND BOARD-LEVEL IT GOVERNANCE CHANGES","This paper presents an empirical study of changes that firms implement in their board-level IT governance (ITG) upon experiencing operational IT failures. Consistent with the separation of oversight from management decisions, board-level ITG is responsible for monitoring managerial IT decisions and policies for controlling IT resources. We expect that operational IT failures indicating inadequacies in board monitoring of controls over IT resources would result in a negative stock market reaction and, in turn, induce firms to improve their board-level ITG. Our expectation is confirmed based on a sample of 110 operational IT failures from U.S. public financial firms. Specifically, our results demonstrate that subsequent to experiencing operational IT failures, firms make improvements to the IT competency level of their boards, and the improvements are proportional to the degree of negative market reaction. However, those improvements are only on the executive side of the board, namely: an increase in the IT experience of internal (executive) directors and an increased turnover rate of CIOs serving on the board. Furthermore, the likelihood of CIO turnover is lower in IT-intensive firms where such turnover could be more disruptive. Our results contribute to understanding the critical connection between operational IT failures and board-level ITG.",MIS QUARTERLY,"OPERATIONAL IT FAILURES, IT VALUE DESTRUCTION, AND BOARD-LEVEL IT GOVERNANCE CHANGES This paper presents an empirical study of changes that firms implement in their board

In [3]:
df = df.drop(['TI','AB'], axis = 1)
df['TI+AB'] = df['TI+AB'].str.lower()

target_dict = {'JOURNAL OF THE ASSOCIATION FOR INFORMATION SYSTEMS': 0,
              'MIS QUARTERLY': 1,
              'INFORMATION SYSTEMS RESEARCH': 2,
              'JOURNAL OF MANAGEMENT INFORMATION SYSTEMS': 3}

df['SO'] = df['SO'].map(target_dict) 
df.head()

,SO,TI+AB
0,0,"the role of business intelligence and communication technologies in organizational agility: a configurational approach this study examines the role that business intelligence (bi) and communication technologies play in how firms may achieve organizational sensing agility, decision making agility, and acting agility in different organizational and environmental contexts. based on the information-processing view of organizations and dynamic capability theory, we suggest a configurational analytic framework that departs from the standard linear paradigm to examine how it's effect on agility is embedded in a configuration of organizational and environmental elements. in line with this approach, we use fuzzy-set qualitative comparative analysis (fsqca) to analyze field survey data from diverse industries. our findings suggest equifinal pathways to organizational agility and the specific boundary conditions of our middle-range theory that determine what role bi and communication technologies play in organizations' achieving organizational agility. we discuss implications for theory and practice and discuss future research avenues."
1,1,"operational it failures, it value destruction, and board-level it governance changes this paper presents an empirical study of changes that firms implement in their board-level it governance (itg) upon experiencing operational it failures. consistent with the separation of oversight from management decisions, board-level itg is responsible for monitoring managerial it decisions and policies for controlling it resources. we expect that operational it failures indicating inadequacies in board monitoring of controls over it resources would result in a negative stock market reaction and, in turn, induce firms to improve their board-level itg. our expectation is confirmed based on a sample of 110 operational it failures from u.s. public financial firms. specifically, our results demonstrate that subsequent to experiencing operational it failures, firms make improvements to the it competency level of their boards, and the improvements are proportional to the degree of negative market reaction. however, those improvements are only on the executive side of the board, namely: an increase in the it experience of internal (executive) directors and an increased turnover rate of cios serving on the board. furthermore, the likelihood of cio turnover is lower in it-intensive firms where such turnover could be more disruptive. our results contribute to understanding the critical connection between operational it failures and board-level itg."
2,1,"on the role of fairness and social distance in designing effective social referral systems online referral systems help firms attract new customers and expand their customer base by leveraging the social relationships of existing customers. we integrate ultimatum game theory, which focuses on fairness, with motivation theories to investigate the effects of social distance and monetary incentives on the performance of three competing designs for online referral systems: rewarding only or primarily the proposer, rewarding only or primarily the responder, and dividing the reward equally or fairly between the proposer and responder. a set of controlled laboratory and randomized field experiments were conducted to test how the fairness of the split of the reward (equal/fair versus unequal/unfair split of the referral bonus) and social distance (small versus large) between the proposer and the responder jointly affect the performance of online referral systems (the proposer sending an offer and the responder accepting the offer). for a large social distance (acquaintances or weak tie relationships), equally splitting the referral bonus results in the best performance. however, for a small social distance (friends or strong tie relationships), an equal split of the referral reward does not improve referral performance, which suggests that under a small social dista

In [4]:
from sklearn.model_selection import train_test_split

features = df['TI+AB']
target = df['SO']

x_train_val, x_test, y_train_val, y_test = \
                    train_test_split(features, target, test_size = 0.20, stratify = target, random_state = 32)

print('x_train_val shape :', x_train_val.shape)
print('x_test shape  :', x_test.shape)
print('y_train_val shape and target counts: {} {}'.format(y_train_val.shape, np.unique(y_train_val, return_counts = True)))
print('y_test shape and target counts: {} {}'.format(y_test.shape, np.unique(y_test, return_counts = True)))

x_train_val shape : (2284,)
x_test shape  : (571,)
y_train_val shape and target counts: (2284,) (array([0, 1, 2, 3], dtype=int64), array([384, 642, 635, 623], dtype=int64))
y_test shape and target counts: (571,) (array([0, 1, 2, 3], dtype=int64), array([ 96, 160, 159, 156], dtype=int64))


In [5]:
import tensorflow
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

# Create a tokenizer -> configured to consider entire unique vocabulary
tokenizer = Tokenizer()

# Can be configured to only take into account the top-N most common words using below
#vocab_count = 10000
#tokenizer = Tokenizer(num_words = vocab_count)

# Build the word index using training & validation data only 
tokenizer.fit_on_texts(x_train_val)

# Computed word index 
word_index = tokenizer.word_index
print('Found {} unique tokens in training data\n'.format(len(word_index)))

# Convert strings into lists of integer indices -> not required here
#sequences = tokenizer.texts_to_sequences(x_train_val)

x_train, x_val, y_train, y_val = \
                    train_test_split(x_train_val, y_train_val, test_size = 0.20, stratify = y_train_val, random_state = 32)

print('x_train shape :', x_train.shape)
print('x_val shape  :', x_val.shape)
print('y_train shape and target counts: {} {}'.format(y_train.shape, np.unique(y_train, return_counts = True)))
print('y_val shape and target counts: {} {}\n'.format(y_val.shape, np.unique(y_val, return_counts = True)))

# Convert input words to one-hot binary representations based on word index created using training and validation data
x_train_val_one_hot = tokenizer.texts_to_matrix(x_train_val, mode='binary')
x_train_one_hot = tokenizer.texts_to_matrix(x_train, mode='binary')
x_val_one_hot = tokenizer.texts_to_matrix(x_val, mode='binary')
x_test_one_hot = tokenizer.texts_to_matrix(x_test, mode='binary')

# One hot encode the target variable -> not encoding, instead using sparse categorical cross entropy loss function
#y_train_one_hot = to_categorical(y_train)
#y_val_one_hot = to_categorical(y_val)
#y_test_one_hot = to_categorical(y_test)

print('x_train_one_hot shape :', x_train_one_hot.shape)
print('x_val_one_hot shape   :', x_val_one_hot.shape)
print('x_test_one_hot shape  :', x_test_one_hot.shape)
print('y_train shape         :', y_train.shape)
print('y_val shape           :', y_val.shape)
print('y_test shape          :', y_test.shape)

Found 14362 unique tokens in training data

x_train shape : (1827,)
x_val shape  : (457,)
y_train shape and target counts: (1827,) (array([0, 1, 2, 3], dtype=int64), array([307, 514, 508, 498], dtype=int64))
y_val shape and target counts: (457,) (array([0, 1, 2, 3], dtype=int64), array([ 77, 128, 127, 125], dtype=int64))

x_train_one_hot shape : (1827, 14363)
x_val_one_hot shape   : (457, 14363)
x_test_one_hot shape  : (571, 14363)
y_train shape         : (1827,)
y_val shape           : (457,)
y_test shape          : (571,)


In [20]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

def train_model(x_train, y_train, x_val, y_val, word_index_len, num_hidden_units_1, num_hidden_units_2, 
                epochs, batch_size, learning_rate):
    model = models.Sequential()
    #model.add(layers.Dense(num_hidden_units_1, activation = 'relu', input_shape = ((word_index_len,)),
    model.add(layers.Dense(num_hidden_units_1, activation = 'relu', input_shape = ((200,)),
                           kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.001)))
                           #kernel_regularizer = regularizers.l1(0.001)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_hidden_units_2, activation = 'relu', 
                           kernel_regularizer = regularizers.l1_l2(l1=0.001, l2=0.001)))
                           #kernel_regularizer = regularizers.l1(0.001)))
    model.add(layers.Dropout(0.5))
    #model.add(layers.Dense(32, activation = 'relu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation = 'softmax'))
    model.summary()
    
    #model.compile(loss = 'categorical_crossentropy', 
    model.compile(loss = 'sparse_categorical_crossentropy', 
                  optimizer = optimizers.RMSprop(lr = learning_rate), 
                  metrics = ['accuracy'])
    
    history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (x_val, y_val))
    return (history, model)

In [22]:
# Tune the hyper parameters -> hidden units in both layers, epochs, batch size, learning rate

#history, model = train_model(x_train_one_hot, y_train_one_hot, x_val_one_hot, y_val_one_hot, 
                              #word_index_len = len(word_index) + 1,  
history, model = train_model(x_train_fixlen, y_train, x_val_fixlen, y_val, word_index_len = len(word_index) + 1, 
#history, model = train_model(x_train_one_hot, y_train, x_val_one_hot, y_val, word_index_len = vocab_count, 
                             num_hidden_units_1 = 64, num_hidden_units_2 = 32, 
                             epochs = 20, batch_size = 128, learning_rate = 0.001)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                12864     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 132       
Total params: 15,076
Trainable params: 15,076
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
15/15 [==============================] - 0s 12ms/step - loss: 2013.3738 - accuracy: 0.2304 - val_loss: 51

In [34]:
# Tune the hyper parameters -> hidden units in both layers, epochs, batch size, learning rate

#history, model = train_model(x_train_one_hot, y_train_one_hot, x_val_one_hot, y_val_one_hot, 
                              #word_index_len = len(word_index) + 1,  
history, model = train_model(x_train_one_hot, y_train, x_val_one_hot, y_val, word_index_len = len(word_index) + 1, 
#history, model = train_model(x_train_one_hot, y_train, x_val_one_hot, y_val, word_index_len = vocab_count, 
                             num_hidden_units_1 = 64, num_hidden_units_2 = 32, 
                             epochs = 20, batch_size = 128, learning_rate = 0.001)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                919296    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 921,508
Trainable params: 921,508
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
15/15 [==============================] - 1s 45ms/step - loss: 4.9097 - accuracy: 0.2715 - val_loss: 2.483

In [7]:
import matplotlib.pyplot as plt

# Plot the Loss curve 
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'bo', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.clf()   # clear figure

# Plot the Accuracy curve 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'bo', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [36]:
test_loss, test_acc = model.evaluate(x_test_one_hot, y_test)
print('Test_Loss: ', test_loss)
print('Test_Accuracy: ', test_acc)

18/18 [==============================] - 0s 5ms/step - loss: 2.0955 - accuracy: 0.3888
Test_Loss:  2.09554123878479
Test_Accuracy:  0.38879159092903137


In [37]:
from sklearn.metrics import confusion_matrix, classification_report

y_test_pred_prob = model.predict(x_test_one_hot)
y_test_pred = np.zeros((len(y_test_pred_prob),))

for i in range(len(y_test_pred_prob)):
    y_test_pred[i] = np.argmax(y_test_pred_prob[i])
    
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_test_pred))
print('\nClassification Report:\n ', classification_report(y_test, y_test_pred, target_names = target_dict))


Confusion Matrix:
 [[22 45 19 10]
 [17 89 36 18]
 [ 8 62 65 24]
 [ 7 57 46 46]]

Classification Report:
                                                      precision    recall  f1-score   support

JOURNAL OF THE ASSOCIATION FOR INFORMATION SYSTEMS       0.41      0.23      0.29        96
                                     MIS QUARTERLY       0.35      0.56      0.43       160
                      INFORMATION SYSTEMS RESEARCH       0.39      0.41      0.40       159
         JOURNAL OF MANAGEMENT INFORMATION SYSTEMS       0.47      0.29      0.36       156

                                          accuracy                           0.39       571
                                         macro avg       0.41      0.37      0.37       571
                                      weighted avg       0.40      0.39      0.38       571



In [423]:
from sklearn.linear_model import LogisticRegressionCV

model_lr = LogisticRegressionCV(cv = 5, max_iter=1000, Cs = np.logspace(-5, 5, 10), random_state = 32, n_jobs = -1)
model_lr.fit(x_train_val_one_hot, y_train_val)

LogisticRegressionCV(Cs=array([1.00000000e-05, 1.29154967e-04, 1.66810054e-03, 2.15443469e-02,
       2.78255940e-01, 3.59381366e+00, 4.64158883e+01, 5.99484250e+02,
       7.74263683e+03, 1.00000000e+05]),
                     cv=5, max_iter=1000, n_jobs=-1, random_state=32)

In [424]:
lr_train_val_acc = model_lr.score(x_train_val_one_hot, y_train_val)
lr_test_acc = model_lr.score(x_test_one_hot, y_test)
print('Train_Val Accuracy: ', lr_train_val_acc)
print('Test Accuracy: ', lr_test_acc)

y_test_pred_lr = model_lr.predict(x_test_one_hot)

print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_test_pred_lr))
print('\nClassification Report:\n ', classification_report(y_test, y_test_pred_lr, target_names = target_dict))

Train_Val Accuracy:  0.9159369527145359
Test Accuracy:  0.4290718038528897

Confusion Matrix:
 [[27 33 19 17]
 [17 71 35 37]
 [ 7 47 62 43]
 [ 8 24 39 85]]

Classification Report:
                                                      precision    recall  f1-score   support

JOURNAL OF THE ASSOCIATION FOR INFORMATION SYSTEMS       0.46      0.28      0.35        96
                                     MIS QUARTERLY       0.41      0.44      0.42       160
                      INFORMATION SYSTEMS RESEARCH       0.40      0.39      0.39       159
         JOURNAL OF MANAGEMENT INFORMATION SYSTEMS       0.47      0.54      0.50       156

                                          accuracy                           0.43       571
                                         macro avg       0.43      0.41      0.42       571
                                      weighted avg       0.43      0.43      0.42       571



In [25]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# We create a tokenizer, configured to only take
# into account the top-1000 most common words
tokenizer = Tokenizer(num_words=5)
# This builds the word index
tokenizer.fit_on_texts(samples)

# This turns strings into lists of integer indices.
sequences = tokenizer.texts_to_sequences(samples)

# You could also directly get the one-hot binary representations.
# Note that other vectorization modes than one-hot encoding are supported!
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


In [26]:
len(one_hot_results[1])

5

In [27]:
sequences

[[1, 2, 3, 4, 1], [1]]

In [52]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(input_train, y_train), (input_test, y_test) = imdb.load_data(num_words=max_features)
print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

print('Pad sequences (samples x time)')
input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)
print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


In [54]:
import tensorflow
import tensorflow.keras as keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 32, input_length = maxlen))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(input_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 19s 124ms/step - loss: 0.5821 - acc: 0.6806 - val_loss: 0.4613 - val_acc: 0.8004
Epoch 2/10
157/157 [==============================] - 19s 119ms/step - loss: 0.3584 - acc: 0.8548 - val_loss: 0.3659 - val_acc: 0.8448
Epoch 3/10
157/157 [==============================] - 20s 130ms/step - loss: 0.2792 - acc: 0.8918 - val_loss: 0.3620 - val_acc: 0.8520
Epoch 4/10
157/157 [==============================] - 21s 133ms/step - loss: 0.2165 - acc: 0.9193 - val_loss: 0.3335 - val_acc: 0.8634
Epoch 5/10
157/157 [==============================] - 19s 120ms/step - loss: 0.1712 - acc: 0.9360 - val_loss: 0.3842 - val_acc: 0.8584
Epoch 6/10
157/157 [==============================] - 20s 125ms/step - loss: 0.1343 - acc: 0.9516 - val_loss: 0.4251 - val_acc: 0.8208
Epoch 7/10
157/157 [==============================] - 20s 127ms/step - loss: 0.0985 - acc: 0.9674 - val_loss: 0.5257 - val_acc: 0.7844
Epoch 8/10
157/157 [==============================] - 1

In [106]:
c = np.logspace(-7, -1, 10)
c

array([1.00000000e-07, 4.64158883e-07, 2.15443469e-06, 1.00000000e-05,
       4.64158883e-05, 2.15443469e-04, 1.00000000e-03, 4.64158883e-03,
       2.15443469e-02, 1.00000000e-01])

In [93]:
x_train = [[1,2,3],[4,5,6],[7,8,9]]
print(x_train)
x_train = [x[::-1] for x in x_train]
print(x_train)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
[[3, 2, 1], [6, 5, 4], [9, 8, 7]]


In [17]:
# Convert input to sequence of fixed length tensor 
inp_max_len = 200
# Convert strings into lists of integer indices sequences 
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Convert lists of integers into a 2D integer tensor of shape `(samples, maxlen)`
x_train_fixlen = preprocessing.sequence.pad_sequences(x_train_seq, maxlen = inp_max_len)
x_val_fixlen = preprocessing.sequence.pad_sequences(x_val_seq, maxlen = inp_max_len)
x_test_fixlen = preprocessing.sequence.pad_sequences(x_test_seq, maxlen = inp_max_len)

print('x_train_fixlen shape :', x_train_fixlen.shape)
print('x_val_fixlen shape   :', x_val_fixlen.shape)
print('x_test_fixlen shape  :', x_test_fixlen.shape)
print('y_train shape        :', y_train.shape)
print('y_val shape          :', y_val.shape)
print('y_test shape         :', y_test.shape)
print('\n')

x_train_fixlen shape : (1827, 200)
x_val_fixlen shape   : (457, 200)
x_test_fixlen shape  : (571, 200)
y_train shape        : (1827,)
y_val shape          : (457,)
y_test shape         : (571,)




In [15]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers
from keras import preprocessing